In [ ]:
import pandas as pd
import altair as alt
import numpy as np

import sqlite3 as sql

import json

# Use to handle large amount of data
# alt.data_transformers.enable('json') 

In [ ]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Data sets used
These are the data sets used in the outline for the examples code.

In [ ]:
cars_url = "https://github.com/byuidatascience/data4missing/raw/master/data-raw/mtcars_missing/mtcars_missing.json" # this gets the cars data from this url
car_db = pd.read_json(cars_url) # uses pandas read_json command to open the data set in pandas format

con = sql.connect("lahmansbaseballdb.sqlite") # allows you to us the sql data set

# Line chat 
how to get a line chat to show up in alt air